# 1. ChatMessageHistory的使用


## 场景1: 记忆存储

In [2]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

print(history.messages)


[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


## 场景2: 对接LLM

In [3]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain.memory import ChatMessageHistory

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)


history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

history.add_user_message("帮我计算1 + 2 + 3 = ?")

response = chat_model.invoke(history.messages)

print(response.content)


按照数学运算顺序，从左到右依次计算：
先计算1 + 2 = 3，再计算3 + 3 = 6。

所以1 + 2 + 3的结果是6。 


# 2. ConversationBufferMemory

- 适用场景：对话轮次较少、依赖完整上下文的场景（如简单的聊天机器）
- 特点：
    - 完整存储对话历史
    - 简单 、 无裁剪 、 无压缩
    - 与 Chains/Models 无缝集成
    - 支持两种返回格式（通过 return_messages 参数控制输出格式）
        - return_messages=True 返回消息对象列表（ List[BaseMessage]
        - return_messages=False （默认） 返回拼接的 纯文本字符串



## 举例1: 以字符串的方式返回存储的信息

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))
 

{'history': 'Human: 你好，我叫小明\nAI: 很高兴认识你\nHuman: 请帮我回到一下1 + 2 * 3 = ?\nAI: 7'}


## 举例2: 以消息列表的方式返回存储的消息

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

print('\n')

print(memory.chat_memory.messages)
 

{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}


[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


## 举例3: 结合大模型、提示词模版的使用(promptTemplate)

In [15]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain.memory import ChatMessageHistory
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{history},
    人类问题:{question},
    回复:
    """
)

memory =  ConversationBufferMemory()

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)





{'question': '我叫小明', 'history': '', 'text': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说哦。 '}


In [16]:
response =  chain.invoke({"question":"我叫什么名字呢？"})

print(response)


{'question': '我叫什么名字呢？', 'history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说哦。 ', 'text': '你叫小明，我记住你的名字啦。要是还有别的话题，也能随时和我分享。 '}


## 举例4: 基于举例3，显示设置memory的key的值

In [17]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{chat_history},
    人类问题:{question},
    回复:
    """
)
memory =  ConversationBufferMemory(memory_key="chat_history")

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)

{'question': '我叫小明', 'chat_history': '', 'text': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 '}


In [19]:
response =  chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'chat_history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 \nHuman: 我叫什么名字呢？\nAI: 你叫小明，你之前已经告诉过我啦。', 'text': '你叫小明呀，我一直都记得呢。 '}


## 举例5: 结合大模型、提示词模版的使用(ChatPromptTemplate)

In [23]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain
from langchain_core. prompts. chat import ChatPromptTemplate
from langchain_core. prompts. chat import MessagesPlaceholder

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个与人类对话的机器人"),
    MessagesPlaceholder(variable_name="history"),
    ("human","问题:{question}")
    
]
)
memory =  ConversationBufferMemory(return_messages=True)

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)

{'question': '我叫小明', 'history': [HumanMessage(content='我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 ', additional_kwargs={}, response_metadata={})], 'text': '你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 '}


In [24]:
response =  chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'history': [HumanMessage(content='我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 ', additional_kwargs={}, response_metadata={}), HumanMessage(content='我叫什么名字呢？', additional_kwargs={}, response_metadata={}), AIMessage(content='你叫小明，刚刚你已经告诉过我啦。', additional_kwargs={}, response_metadata={})], 'text': '你叫小明，刚刚你已经告诉过我啦。'}


# 3. ConversationChain的使用

## 举例1: 以PromptTemplate为例

In [31]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. chains. conversation. base import ConversationChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{history},
    人类问题:{input},
    回复:
    """
)
# memory =  ConversationBufferMemory(return_messages=True)
# 
# chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

# 创建ConversationChain的实例
chain = ConversationChain(llm=chat_model, prompt=prompt_template)

response =  chain.invoke({"input":"我叫小明"})

print(response)

{'input': '我叫小明', 'history': '', 'response': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 '}


In [33]:
response =  chain.invoke({"input":"我叫什么名字呢？"})
print(response)

{'input': '我叫什么名字呢？', 'history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 ', 'response': '你叫小明，之前你已经告诉过我啦。'}
